# Cassava Plant Disease Classification
![](https://cff2.earth.com/uploads/2020/11/11065615/shutterstock_12823926312-scaled.jpg)

In this tutorial, I will run you through an end-to-end solution for classifying 4 different types of diseases found in Cassava Plants through Tensorflow & Keras. 

The following topics will be covered:
1. Exploratory Data Analysis
2. Image Augmentation
3. Model Building and Selection
4. Stratified K-Fold Cross Validation
5. Model Ensembling and Inference

# Importing Libraries and Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
import os
import cv2
import sys
from pylab import rcParams
from PIL import Image
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Input, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import InceptionV3, Xception
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy) #shortens training time by 2x

First, let's take a look at our dataframe.

In [ ]:
df_train = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
df_train.head()

In [ ]:
df_train["label"] = df_train["label"].astype(str) #convert to str as we want to use Categorical Cross Entropy (CCE) later on
df_train.info()

# Exploratory Data Analysis

Now, we will take a look at the distribution of our data.

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(10,8))
sns.countplot(df_train["label"], edgecolor="black", palette="mako")

From here, we can see that we have a highly unbalanced dataset. We have the largest number of samples for label 3, Cassava Mosiac Disease (CMD), and the fewest number of samples for label 0, Cassava Bacterial Blight (CBB). Let's visualize how some of these images look like next.

### Label 0: Cassava Bacterial Blight (CBB)

In [ ]:
path = "../input/cassava-leaf-disease-classification/train_images/"
df0 = df_train[df_train["label"] == "0"]
files = df0["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

### Label 1: Cassava Brown Streak Disease (CBSD)

In [ ]:
df1 = df_train[df_train["label"] == "1"]
files = df1["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

### Label 2: Cassava Green Mottle (CGM)

In [ ]:
df2 = df_train[df_train["label"] == "2"]
files = df2["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

### Label 3: Cassava Mosiac Disease (CMD)

In [ ]:
df3 = df_train[df_train["label"] == "3"]
files = df3["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

### Label 4: Healthy

In [ ]:
df4 = df_train[df_train["label"] == "4"]
files = df4["image_id"].sample(3).tolist()

plt.figure(figsize=(15,5))
index = 0
for file in files:
    image = Image.open(path + file)
    plt.subplot(1, 3, index + 1)
    plt.imshow(image)
    plt.axis("off")
    index += 1

plt.show()

Note that the images in this training set are "noisy". This means that there are some images that are mislabelled and hence will affect predictive performance. One of the basic ways to mitigate this is to use label smoothing on our loss metric, as you will see later on.

# Image Augmentation (Tensorflow)
Image augmentation is important as, with all Machine Learning problems, we do not want the model to "memorise" the training set. In this case, we do not want our algorithm to memorise features specific to the training images such as the orientation, position or shade of the cassava plant. Therefore, we need to augment the images so that the model is able to generalise well and not overfit to the training set.

One of the ways we can perform image augmentation is through Tensorflow preprocessing layers. They provide basic augmentations such as cropping, flipping and rotating.

In [ ]:
batch_size=16
image_size=300

input_shape = (image_size, image_size, 3)
target_size = (image_size, image_size)

In [ ]:
img_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomCrop(image_size, image_size),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.25),
        tf.keras.layers.experimental.preprocessing.RandomZoom((-0.25, 0.25), (-0.25, 0.25)),
    ])

Now let's see how an image looks like pre-augmentation and post-augmentation.

In [ ]:
path = "../input/cassava-leaf-disease-classification/train_images/"
files = df_train["image_id"].tolist()
file = random.choice(files)
image = Image.open(path + file)
plt.imshow(image)
plt.axis("off")
plt.show()

Here are 9 possible outcomes after augmentation.

In [ ]:
image = tf.expand_dims(np.array(image), 0)

plt.figure(figsize=(14, 14))
for i in range(9):
    augmented_image = img_augmentation(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

# Image Augmentation (Albumentations)
I will now show you how to perform image augmentations through Albumentation (an external image augmentation library with much more functionality) through both ImageDataGenerator. We will use a tool called ImageDataAugmentor (big thanks to mjkvaak at github) that allows us to do this.

In [ ]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

train_augmentations = A.Compose([
            A.RandomCrop(image_size, image_size, p=1),
            A.CoarseDropout(p=0.5),
            A.Cutout(p=0.5),
            A.Flip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.5, hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2),
            A.RandomBrightnessContrast(p=0.5, brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2)),
            A.ToFloat()
            ], p=1)

val_augmentations = A.Compose([
                A.CenterCrop(image_size, image_size, p=1),
                A.ToFloat()
                ], p=1)

In [ ]:
def TFDataGenerator(train_set, val_set):
    
    train_generator = ImageDataAugmentor(augment=train_augmentations)
    val_generator = ImageDataAugmentor(augment=val_augmentations)
    
    train_datagen = train_generator.flow_from_dataframe(
                  dataframe = train_set,
                  directory='../input/cassava-leaf-disease-classification/train_images',
                  x_col='image_id',
                  y_col='label',
                  target_size=target_size,
                  batch_size=batch_size,
                  shuffle=True,
                  class_mode='categorical',
                  seed=2020)

    val_datagen = val_generator.flow_from_dataframe(
                dataframe = val_set,
                directory='../input/cassava-leaf-disease-classification/train_images',
                x_col='image_id',
                y_col='label',
                target_size=target_size,
                batch_size=batch_size,
                shuffle=False,
                class_mode='categorical',
                seed=2020)
    
    return train_datagen, val_datagen

Now let's see how the generated images look like post-augmentation for our training set.

In [ ]:
train_set = df_train.iloc[:10]
val_set = df_train.iloc[-10:]

train_datagen, val_datagen = TFDataGenerator(train_set, val_set)

In [ ]:
train_images, _ = next(train_datagen)

plt.figure(figsize=(14, 14))
for i in range(9):
    image = train_images[i]
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.axis("off")

plt.show()

And now our validation set:

In [ ]:
val_images, _ = next(val_datagen)

plt.figure(figsize=(14, 14))
for i in range(9):
    image = val_images[i]
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.axis("off")

plt.show()

# Model Building and Selection
Earlier on, I mentioned that to counter mislabelling, we would implement label smoothing on our loss metric (in this case, categorical cross-entropy). Here's how it works:

When we apply the cross-entropy loss to a classification task, we’re expecting true labels to be 1, while the others to be 0. In other words, we are sure that the labels are 100% correct. However, we already know that ***they are not!***

One way to circumvent this is to lower the confidence level of our labels. So instead of being 100% confident (or sure) that our labels are correct, we can be 90% or even 80% confident instead. This allows the model to "learn" that some the labels are not correct and make better predictions.

Originally, if an image is labelled "0" (Cassava Bacterial Blight Disease), the resulting one-hot label would be this: [1, 0, 0, 0, 0]. However, after applying label smoothing, then we get new one-hot labels according to this formula:

new_onehot_labels = onehot_labels * (1 - label_smoothing) + label_smoothing / num_classes

In our case, with a label smoothing value of 0.2, 

new_onehot_labels 

= [1, 0, 0, 0, 0] * (1 - 0.2) + 0.2 / 5

= [1, 0, 0, 0, 0] * 0.8 + 0.04

= [0.84, 0.04, 0.04, 0.04, 0.04]
                  
This means that we are ~84% confident in our labels!

In [ ]:
def create_Inception():
    base_model = InceptionV3(include_top=False, weights="imagenet", input_shape=input_shape)

    # Rebuild top
    inputs = Input(shape=input_shape)

    model = base_model(inputs)
    pooling = GlobalAveragePooling2D()(model)
    dropout = Dropout(0.2)(pooling)

    outputs = Dense(5, activation="softmax", name="dense", dtype='float32')(dropout)

    # Compile
    inception = Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2, from_logits=True)

    inception.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return inception

In [ ]:
def create_Xception():
    base_model = Xception(include_top=False, weights="imagenet", input_shape=input_shape)

    # Rebuild top
    inputs = Input(shape=input_shape)

    model = base_model(inputs)
    pooling = GlobalAveragePooling2D()(model)
    dropout = Dropout(0.2)(pooling)

    outputs = Dense(5, activation="softmax", name="dense", dtype='float32')(dropout)

    # Compile
    xception = Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2, from_logits=True)

    xception.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return xception

We will be using two model architectures, InceptionV3 and Xception, for our model ensembling. I chose Stochastic Gradient Descent (SGD) with momentum as it generalizes better. Additionally, I enabled Nesterov momentum as it (theoretically) converges to a minimum more quickly. To understand the math behind the two implementations of SGD with momentum, refer to this: https://stats.stackexchange.com/questions/179915/whats-the-difference-between-momentum-based-gradient-descent-and-nesterovs-acc

![](https://i.stack.imgur.com/YrpGA.png)

# Training with Stratified K-Fold Cross Validation
We need to train our models with k-fold cross validation to ensure that our model isn't overfitting on the training data (or rather, specific aspects of the training data). Stratified k-fold cv ensures that we have the same class representation in every fold. Here's a brief intuition:

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0041.png)

In our case, we take 90% of the train images for training and 10% for testing. However, this split can be arbitary. Moreover, we saw earlier on that the data is imbalanced. Typically, the class with the most images will be the easiest to predict as the model is able to train much better with more data. Hence, splitting the dataset randomly can cause huge variance in the model's performance. For example, on some splits, the model might get a validation subset that is easy to predict.. On others, it might get a difficult subset to predict.

By using stratified cross validation, we ensure that a different subset of the data is split into training and test sets every time. Additionally, we ensure that class representation remains the same for all subsets, thereby allocating equal difficulty in all subsets.

We will only do a stratified 3-fold cv this time due to time constraints.

In [ ]:
fold_number = 0
n_splits = 3
epochs = 8

tf.keras.backend.clear_session()
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.1, random_state=2020)
for train_index, val_index in sss.split(df_train["image_id"], df_train["label"]):
    train_set = df_train.loc[train_index]
    val_set = df_train.loc[val_index]
    train_datagen, val_datagen = TFDataGenerator(train_set, val_set)
    model = create_Inception()
    print("Training fold no.: " + str(fold_number+1))

    model_name = "inception "
    fold_name = "fold.h5"
    filepath = model_name + str(fold_number+1) + fold_name
    callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.2),
                 EarlyStopping(monitor='val_loss', patience=3),
                 ModelCheckpoint(filepath=filepath, monitor='val_loss', save_best_only=True)]

    history = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
    fold_number += 1
    if fold_number == n_splits:
        print("Training finished!")

In [ ]:
fold_number = 0
n_splits = 3
epochs = 8

tf.keras.backend.clear_session()
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.1, random_state=2020)
for train_index, val_index in sss.split(df_train["image_id"], df_train["label"]):
    train_set = df_train.loc[train_index]
    val_set = df_train.loc[val_index]
    train_datagen, val_datagen = TFDataGenerator(train_set, val_set)
    model = create_Xception()
    print("Training fold no.: " + str(fold_number+1))

    model_name = "xception "
    fold_name = "fold.h5"
    filepath = model_name + str(fold_number+1) + fold_name
    callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.2),
                 EarlyStopping(monitor='val_loss', patience=3),
                 ModelCheckpoint(filepath=filepath, monitor='val_loss', save_best_only=True)]

    history = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
    fold_number += 1
    if fold_number == n_splits:
        print("Training finished!")

# Model Ensembling and Inference
This is the part where you should take the code below into a separate inference notebook with minor code changes to the filepaths. Reason being that we needed the internet enabled to install ImageDataAugmentor but internet access is not allowed in this competition. 

In [ ]:
models = []
for i in range(n_splits):
    inception = load_model("./inception " + str(i+1) + "fold.h5")
    models.append(inception)
    
for i in range(n_splits):
    xception = load_model("./xception " + str(i+1) + "fold.h5")
    models.append(xception)

In [ ]:
ss = pd.read_csv(os.path.join('../input/cassava-leaf-disease-classification', "sample_submission.csv"))
preds = []
results = []

for image_id in ss.image_id:
    image = Image.open(os.path.join('../input/cassava-leaf-disease-classification', "test_images", image_id))
    image = image.resize((image_size, image_size))
    image = np.expand_dims(image, axis = 0)
    for model in models:
        preds.append(np.argmax(model.predict(image)))
    res = max(set(preds), key = preds.count)
    results.append(res)

ss['label'] = results
ss.to_csv('submission.csv', index = False)

And done! Please upvote this notebook if you liked it, it inspires me to make more quality notebooks. Thank you!